# Uplift Modeling with EconML using MovieLens 1M
This notebook downloads MovieLens 1M data, simulates treatment and renewal outcomes, introduces missing data, imputes values, and trains S-, T-, and X-Learners using different base learners.

In [1]:
!git branch

fatal: not a git repository (or any of the parent directories): .git


In [2]:
!pip install econml scikit-learn pandas numpy wget

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 13.5 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=7a8cad00d0d74397d8561bf2c1d173d03f6bc5291cc5d612a37eadca044fbe35
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: numpy
    Found existing install

In [3]:
# Download and extract MovieLens 1M dataset
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

--2025-05-16 17:11:26--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  8.79MB/s    in 0.6s    

2025-05-16 17:11:27 (8.79 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   


In [5]:
import numpy as np

In [6]:
!pip install pandas

In [8]:
import pandas as pd

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [9]:
!pip uninstall -y pandas numpy

Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


In [10]:
!pip install pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 42.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
econml 0.15.1 requires numpy<2, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.


In [2]:
!pip  uninstall econml scikit-learn pandas numpy

Found existing installation: econml 0.15.1
Uninstalling econml-0.15.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/econml-0.15.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/econml/*
Proceed (Y/n)? Y
  Successfully uninstalled econml-0.15.1
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_learn-1.5.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/sklearn/*
Proceed (Y/n)? Y
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pandas-2.2.3.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pandas/*
Proceed (Y/n)? y
  Successfully uninstalled pandas-2.2.3
Found existing installation: numpy 2.2.5
Uninstalling numpy-2.2.5:
  Would remove:
    /usr

In [3]:
!pip  install econml scikit-learn pandas numpy

  Using cached econml-0.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (38 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached econml-0.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages t

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from econml.metalearners import SLearner, TLearner, XLearner

In [6]:
# Download and extract MovieLens 1M dataset
# The wget and unzip commands appear to be working correctly based on your output.
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

# Add checks to verify if the directory and file exist
import os

# Correct the path to reflect the nested directory structure
if os.path.exists('ml-1m/ml-1m/ratings.dat'):
    print("ml-1m/ml-1m/ratings.dat found. Proceeding to load data.")
else:
    print("Error: ml-1m/ml-1m/ratings.dat not found. Please check the extraction path.")
    # If the file is still not found after correcting the path, there might be
    # a deeper issue with the unzip process or disk.
    # import sys
    # sys.exit(1) # Uncomment to exit the notebook execution if the file is not found

--2025-05-16 17:16:38--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.3’

ml-1m.zip.3         100%[===================>]   5.64M  9.75MB/s    in 0.6s    

2025-05-16 17:16:39 (9.75 MB/s) - ‘ml-1m.zip.3’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   
ml-1m/ml-1m/ratings.dat found. Proceeding to load data.


In [8]:
# Load data
ratings = pd.read_csv('ml-1m//ml-1m/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [9]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# Feature creation
np.random.seed(42)
df['WatchTime'] = df['Rating'] * np.random.uniform(15, 30, size=len(df))
df['TenureMonths'] = (df['Timestamp'] - df['Timestamp'].min()) // (60*60*24*30)
user_features = df.groupby('UserID').agg({
    'WatchTime': 'sum',
    'MovieID': 'nunique',
    'TenureMonths': 'max',
    'Age': 'first',
    'Occupation': 'first'
}).rename(columns={'WatchTime': 'TotalWatchTime', 'MovieID': 'UniqueMovies'})

In [ ]:
# Introduce and impute missing data
user_features.loc[user_features.sample(frac=0.1).index, 'TotalWatchTime'] = np.nan
user_features.loc[user_features.sample(frac=0.1).index, 'TenureMonths'] = np.nan
user_features['TotalWatchTime'].fillna(user_features['TotalWatchTime'].median(), inplace=True)
user_features['TenureMonths'].fillna(user_features['TenureMonths'].median(), inplace=True)

In [ ]:
# Simulate treatment and renewal
user_features['treatment'] = np.random.binomial(1, 0.5, size=len(user_features))
engaged = user_features['TotalWatchTime'] > user_features['TotalWatchTime'].median()
base_rate = 0.2
uplift = 0.15 * ((user_features['treatment'] == 1) & engaged).astype(float)
user_features['renewed'] = np.random.binomial(1, base_rate + uplift)
X = user_features[['TenureMonths', 'TotalWatchTime', 'UniqueMovies']]
T = user_features['treatment'].values
Y = user_features['renewed'].values

In [ ]:
# Split data
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

In [ ]:
# Hyperparameter tuning
lr_grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid={'C': [0.01, 0.1, 1, 10]}, cv=3)
lr_grid.fit(X_train, Y_train)
best_lr = lr_grid.best_estimator_

rf_random = RandomizedSearchCV(RandomForestRegressor(random_state=42),
    param_distributions={'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    n_iter=4, cv=3, random_state=42)
rf_random.fit(X_train, Y_train)
best_rf = rf_random.best_estimator_

gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42),
    param_grid={'n_estimators': [100, 150], 'learning_rate': [0.05, 0.1]}, cv=3)
gb_grid.fit(X_train, Y_train)
best_gb = gb_grid.best_estimator_

In [ ]:
# Train learners
s_learner = SLearner(learner=best_lr)
t_learner = TLearner(models=best_rf)
x_learner = XLearner(models=best_gb)
s_learner.fit(Y_train, T_train, X=X_train)
t_learner.fit(Y_train, T_train, X=X_train)
x_learner.fit(Y_train, T_train, X=X_train)
s_te = s_learner.effect(X_test)
t_te = t_learner.effect(X_test)
x_te = x_learner.effect(X_test)
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head()